In [2]:
import numpy as np
import pandas as pd
import math as m
import sympy as sym
import random as rand
import matplotlib.pyplot as plt
import cmath as cm

In [19]:
theta1, theta2 = sym.symbols('theta1 theta2')


def Creating_states( c = [0, m.pi/2] , priors = [.5,.5], input_type = 'theta', Abstract = False):    # coeff list like [a0,a1,b0,b1]
    
    if Abstract == True:
        theta1, theta2 = sym.symbols('theta1 theta2')
        a0, a1 = sym.cos(theta1), sym.sin(theta1)
        b0, b1 = sym.cos(theta2), sym.sin(theta2)
    else:
        theta1 , theta2 = c[0], c[1]
        a0, a1 = sym.cos(theta1), sym.sin(theta1)
        b0, b1 = sym.cos(theta2), sym.sin(theta2)
    
    psi0, psi1 = [a0,a1], [b0,b1]      # defining states
    
    
    psi0psi0, psi1psi1 = [psi0[0]*psi0[0], (np.sqrt(2)*psi0[0]*psi0[1]) , psi0[1]*psi0[1]],  [psi1[0]*psi1[0], (np.sqrt(2)*psi1[0]*psi1[1]), psi1[1]*psi1[1]] # @@@ the square rooting is simplistic. see notebook for better
    
    # vec_psi0psi0, vec_psi1psi1 = np.array(psi0psi0), np.array(psi1psi1)     # np array conversion 
    # rho = priors[0]*np.outer(vec_psi0psi0, vec_psi0psi0)+ priors[1]*np.outer(vec_psi1psi1, vec_psi1psi1)     # theoretical rho
    
    return([psi0,psi1], [psi0psi0,psi1psi1])

x = Creating_states([0,-m.pi/4], Abstract=1)[1]

y = [i.subs({theta1 : 0, theta2 : -m.pi/4}) for i in x[1]]
y
# Creating_states([0,-.927])[1]

# print(1.41*.8*.6)

[0.500000000000000, -0.707106781186548, 0.500000000000000]

In [14]:
psi0 = Creating_states(Abstract=False)[0]
psi1 = Creating_states(Abstract=False)[1]

psi0

[0.7071067811865475, 0.7071067811865475]

In [15]:
psi0sq = []
psi1sq = []
[[psi0sq.append(i*j) for i in psi0] for j in psi0]
[[psi1sq.append(i*j) for i in psi1] for j in psi1]   # creating list of coeff for squared states (intermeditate step)

psi0psi0 = [psi0sq[0], (psi0sq[1]**2+psi0sq[2]**2)**(.5), psi0sq[3]]        # the second element is the normalized coefficient due to the addition of states (01+10). See OneNote writeup for details
psi1psi1 = [psi1sq[0], (psi1sq[1]**(2)+psi1sq[2]**(2))**(.5), psi1sq[3]]            # symmetric states in basis (|00>, normalized version of (|01>+|10>),|11>)

# psi0psi0
# sum([i**2 for i in psi0psi0])   # adding to 1 now.

In [16]:
#creating the density matrix rho
# rho is 1/2*(|psi0psi0Xpsi0psi0|+|psi1psi1Xpsi1psi1|)
# findiing the conjugate vectors 
vec_psi0psi0 = sym.Matrix(psi0psi0)     # convertying to sympy vectors for easy matrix multiplication
vec_psi1psi1 = sym.Matrix(psi1psi1)

rho = (1/2)*(vec_psi0psi0*vec_psi0psi0.T  + vec_psi1psi1*vec_psi1psi1.T)       # density matrix with priors 1/2 each.
rho

Matrix([
[            0.625, 0.176776695296637,             0.125],
[0.176776695296637,              0.25, 0.176776695296637],
[            0.125, 0.176776695296637,             0.125]])

In [17]:
# creating the SIC POVM matrices
w = m.e**((2/3)*m.pi*(1j))
POVM_vec = (1/(2**.5))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))
POVM_elts = [(1/3)*np.outer(np.conjugate(POVM_vec[i]),POVM_vec[i]) for i in range(len(POVM_vec))]
# sum([np.trace(POVM_elts[i]) for i in range(9)])`# trace sum checks out`

In [18]:
# prob_vec = [(POVM_elts[i]*rho).trace() for i in range(len(POVM_elts))]
prob_vec =  [np.trace(np.dot(POVM_elts[i],rho)) for i in range(9)] #[(POVM_elts[i]*rho).trace() for i in range(len(POVM_elts))]
# prob_vec
sum(prob_vec)        # prob not adding to 1?
# print(prob_vec)


0.999999999999999

In [19]:
# # verifying that the trace still holds to be the same, equalt to .111 for this weird POVM element.
# c = .00308641975308642*2.71828182845905**(4.18879020478639*(sym.I))

# p = cm.polar(c)
# cm.rect(p[0], p[1])

# r = .00617-.0015
# i = .0026

# ab = (r**2+i**2)**(.5)
# 2*ab # verified

In [20]:
N = 10000       # number of measurements  

POVM_dir_symbols = ['d1','d2','d3','d4','d5','d6','d7','d8','d9']       # symbols to indicate collapsed direction
#prob distribution is simply the corresponding elements of the prob_vec
collapse_dir_vec = rand.choices(POVM_dir_symbols, weights=prob_vec, k = N)   # choosing collapse directions with weights for N trials

nj_vec = [collapse_dir_vec.count(f'd{i+1}') for i in range(9)]         # working babieee
pj_num_vec = [i/N for i in nj_vec]         # working babieee
([i*N for i in prob_vec] , nj_vec)   # comparing the theoretical and experimental collapse prob.

([35.7443490112103,
  833.333333333333,
  869.077682344544,
  919.627825494394,
  1458.33333333333,
  1752.96115882773,
  919.627825494394,
  1458.33333333333,
  1752.96115882773],
 [40, 843, 868, 909, 1418, 1733, 920, 1524, 1745])

In [21]:
M = [[np.trace(np.dot(POVM_elts[i],POVM_elts[j])) for i in range(len(POVM_elts))] for j in range(len(POVM_elts))]     # creating M matrix using POVM definition


In [22]:
u_0 = [1/3 for i in range(9)]           # cerating u_0 vector, to create the inverse matrix
M_inv = 3*np.outer(u_0,u_0) + 12*(np.eye(9) - np.outer(u_0,u_0))        # creating the inverse matrix
# np.trace(np.dot(M,M_inv))       # gives identity, checks out
r_vec = np.dot(M_inv,pj_num_vec)


In [23]:
rho_num_list = [r_vec[i]*POVM_elts[i] for i in range(len(POVM_elts))]
[i for i in rho_num_list]
3

3

In [24]:
rho_num = np.zeros_like(rho_num_list[0])

# Loop over the matrices and sum them element-wise
for matrix in rho_num_list:
    rho_num = np.add(rho_num, matrix)

rho_num   

array([[0.6262+0.00000000e+00j, 0.1742+2.07846097e-03j,
        0.1256-1.83597386e-02j],
       [0.1742-2.07846097e-03j, 0.243 +3.17604419e-18j,
        0.1749+1.90525589e-03j],
       [0.1256+1.83597386e-02j, 0.1749-1.90525589e-03j,
        0.1308+2.65805330e-18j]])

In [25]:
rho # printing the theoretical density matrix and below the numerically reconstructed one

Matrix([
[            0.625, 0.176776695296637,             0.125],
[0.176776695296637,              0.25, 0.176776695296637],
[            0.125, 0.176776695296637,             0.125]])

In [26]:
np.matrix(rho_num)  

matrix([[0.6262+0.00000000e+00j, 0.1742+2.07846097e-03j,
         0.1256-1.83597386e-02j],
        [0.1742-2.07846097e-03j, 0.243 +3.17604419e-18j,
         0.1749+1.90525589e-03j],
        [0.1256+1.83597386e-02j, 0.1749-1.90525589e-03j,
         0.1308+2.65805330e-18j]])

In [27]:
rho 
del_rho = rho_num-rho
del_rho

Matrix([
[                         0.00119999999999987,  -0.00257669529663684 + 0.00207846096908257*I,  0.000600000000000017 - 0.0183597385602301*I],
[-0.00257669529663684 - 0.00207846096908257*I, -0.00700000000000006 + 3.17604418755426e-18*I, -0.00187669529663684 + 0.00190525588832576*I],
[ 0.000600000000000017 + 0.0183597385602301*I,  -0.00187669529663684 - 0.00190525588832576*I, 0.00579999999999997 + 2.65805330262845e-18*I]])

In [28]:
# evects = del_rho.eigenvects()
eigensystem = del_rho.eigenvects()

#separating eigenvalues of del rho
evects_0 = []
evals_0 = []

for i in range(3):
    evals_0.append(eigensystem[i][0])
    evects_0.append(eigensystem[i][-1])

evals = [sym.re(i) for i in evals_0]
norm_del_rho = max([abs(i) for i in evals])     # picking out the largest evalue

evals

[-0.00752496529537751, -0.0150822120132936, 0.0226071773086709]

In [29]:
#eigenvalues of rho_num
eigensystem_num = sym.Matrix(rho_num).eigenvects()
evects_num = []
evals_num = []

for i in range(3):
    evals_num.append(eigensystem_num[i][0])
    evects_num.append(eigensystem_num[i][-1])

# evals = [sym.re(i) for i in evals_0]
evals_num = [sym.re(i) for i in evals_num]

zero_eval_rho_num = min(evals_num)                          # picking the minimum evalue 
zero_eval_index = evals_num.index(zero_eval_rho_num)        # finding index so we can pick the corresponding evector
perp_evect_rho_num = evects_num[zero_eval_index]

c00 = perp_evect_rho_num[0][0]
c01 = perp_evect_rho_num[0][1]
c11 = perp_evect_rho_num[0][2]         # the coefficients of the xi vector

perp_evect_rho_num



[Matrix([
 [-0.00296868476851878 - 0.0344054942133637*I],
 [    0.589477331704725 + 0.011959532096648*I],
 [  -0.806519892761224 + 0.0265811297240971*I]])]

In [30]:
Quadratic_Form = lambda a,b,c : [(-b+ ((b**2-4*a*c)**(.5)))/(2*a), (-b- ((b**2-4*a*c)**.5))/(2*a)]

In [31]:
[sym.simplify(Quadratic_Form(c00,c01,c11)[0]), sym.simplify(Quadratic_Form(c00,c01,c11)[1])]

[0.906227812679038 - 8.48842220771291*I + (-0.719446754247216 + 8.33800927689144*I)*(1 - 0.2890691051273*I)**0.5,
 0.906227812679038 + (0.719446754247216 - 8.33800927689144*I)*(1 - 0.2890691051273*I)**0.5 - 8.48842220771291*I]